<a href="https://colab.research.google.com/github/geraw/ProvengoDemo/blob/main/ProvegoTutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [29]:
!wget -nc https://downloads.provengo.tech/unix-dist/deb/Provengo-deb.deb
!apt-get install ./Provengo-deb.deb

'wget' is not recognized as an internal or external command,
operable program or batch file.
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
%pip install flask

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


# System Under Test

## Setting up Flask app

In [31]:
from flask import Flask, request, jsonify
import threading
import time
import os
import sys
import json
import importlib
import signal


# Create Flask app
app = Flask(__name__)
#run_with_ngrok(app)  # Allows access from outside

# In-memory data store (replace with a database in a real application)
users = []
loans = []
holds = []
books = {}


## A route for resetting the database

In [32]:
# A rute for testing that resets the database and loads initial data
@app.route('/reset', methods=['POST'])
def reset_database():
    global users, loans, holds, books
    
    # Clear all data
    users = []
    loans = []
    holds = []
    books = {}
    
    # If initial data is provided, load it
    if request.is_json:
        data = request.get_json()
        if 'users' in data:
            users.extend(data['users'])
        if 'loans' in data:
            loans.extend(data['loans'])
        if 'holds' in data:
            holds.extend(data['holds'])
        if 'books' in data:
            books.update(data['books'])
    
    return jsonify({
        'message': 'Database reset',
        'status': {
            'users': len(users),
            'loans': len(loans),
            'holds': len(holds),
            'books': len(books)
        }
    }), 200




## User routes

In [33]:
# --- User Routes ---
@app.route('/users', methods=['POST'])
def add_user():
    user = request.get_json()

    if 'id' not in user:
        print("Error: Attempt to add user without id")
        return jsonify({'error': 'user id is required'}), 400
    if  user.get('id') in [u.get('id') for u in users]:
        print("Error: Attempt to add duplicate user")
        return jsonify({'error': 'User already exists'}), 400
    else:
      users.append(user)
      return jsonify({'message': 'User Added', 'user': user}), 201

@app.route('/users/<int:user_id>', methods=['DELETE'])
def delete_user(user_id):
    global users
    users = [user for user in users if user.get('id') != user_id]
    return jsonify({'message': 'User deleted'}), 200

@app.route('/users', methods=['GET'])
def search_users():
    query = request.args.get('q', '').lower()
    results = [user for user in users if query in str(user).lower()] if query else users
    return jsonify(results)



## Loan routes

In [34]:
# --- Loan Routes ---
@app.route('/loans', methods=['POST'])
def add_loan():
    loan = request.get_json()
    loans.append(loan)
    return jsonify({'message': 'Loan added', 'loan': loan}), 201

@app.route('/loans/<int:loan_id>', methods=['DELETE'])
def delete_loan(loan_id):
    global loans
    loans = [loan for loan in loans if loan.get('id') != loan_id]
    return jsonify({'message': 'Loan deleted'}), 200

@app.route('/loans', methods=['GET'])
def search_loans():
    query = request.args.get('q', '').lower()
    results = [loan for loan in loans if query in str(loan).lower()] if query else loans
    return jsonify(results)



## Hold routes

In [35]:
# --- Hold Routes ---
@app.route('/holds', methods=['POST'])
def add_hold():
    hold = request.get_json()
    holds.append(hold)
    return jsonify({'message': 'Hold added', 'hold': hold}), 201

@app.route('/holds/<int:hold_id>', methods=['DELETE'])
def delete_hold(hold_id):
    global holds
    holds = [hold for hold in holds if hold.get('id') != hold_id]
    return jsonify({'message': 'Hold deleted'}), 200

@app.route('/holds', methods=['GET'])
def search_holds():
    query = request.args.get('q', '').lower()
    results = [hold for hold in holds if query in str(hold).lower()] if query else holds
    return jsonify(results)



## Book routes

In [36]:
# --- Book Routes ---
@app.route('/books', methods=['POST'])
def add_book():
    book = request.get_json()

    if 'id' not in book:
        print("Error: Attempt to add book without id")
        return jsonify({'error': 'book id is required'}), 400
    if book.get('id') in books:
        print("Error: Attempt to add duplicate book")
        return jsonify({'error': 'Book already exists'}), 400
    else:
        books[book['id']] = book
        return jsonify({'message': 'Book Added', 'book': book}), 201

@app.route('/books/<book_id>', methods=['DELETE'])
def delete_book(book_id):
    if book_id in books:
        del books[book_id]
        return jsonify({'message': 'Book deleted'}), 200
    return jsonify({'error': 'Book not found'}), 404

@app.route('/books', methods=['GET'])
def search_books():
    query = request.args.get('q', '').lower()
    results = [book for book in books.values() if query in str(book).lower()] if query else list(books.values())
    return jsonify(results)

@app.route('/books/<book_id>', methods=['GET'])
def get_book(book_id):
    if book_id in books:
        return jsonify(books[book_id])
    return jsonify({'error': 'Book not found'}), 404

## Run the server

In [ ]:
import socket
import random

host = socket.gethostbyname(socket.gethostname())
print(f"{host=}")

port = random.randint(1024, 65535)
url = f"http://{host}:{port}"

threading.Thread(target=app.run, kwargs={'host':host,'port':port}).start()


host='192.168.56.1'
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://192.168.56.1:25221
Press CTRL+C to quit
192.168.56.1 - - [15/Mar/2025 20:57:43] "POST /reset HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 20:57:43] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 20:57:43] "POST /books HTTP/1.1" 400 -
192.168.56.1 - - [15/Mar/2025 20:57:43] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 20:57:43] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 20:57:43] "GET /books HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 20:57:43] "GET /books HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 20:57:43] "GET /books HTTP/1.1" 200 -


Error: Attempt to add duplicate book


192.168.56.1 - - [15/Mar/2025 20:58:02] "POST /reset HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 20:58:02] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 20:58:02] "POST /books HTTP/1.1" 400 -
192.168.56.1 - - [15/Mar/2025 20:58:02] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 20:58:02] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 20:58:02] "GET /books HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 20:58:02] "GET /books HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 20:58:02] "GET /books HTTP/1.1" 200 -


Error: Attempt to add duplicate book


...

Error: Attempt to add duplicate book


192.168.56.1 - - [15/Mar/2025 20:59:22] "POST /reset HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 20:59:23] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 20:59:23] "POST /books HTTP/1.1" 400 -
192.168.56.1 - - [15/Mar/2025 20:59:23] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 20:59:23] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 20:59:23] "GET /books HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 20:59:23] "GET /books HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 20:59:23] "GET /books HTTP/1.1" 200 -


Error: Attempt to add duplicate book


192.168.56.1 - - [15/Mar/2025 21:00:38] "POST /reset HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 21:00:38] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 21:00:38] "POST /books HTTP/1.1" 400 -
192.168.56.1 - - [15/Mar/2025 21:00:38] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 21:00:38] "POST /books HTTP/1.1" 201 -
192.168.56.1 - - [15/Mar/2025 21:00:38] "GET /books HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 21:00:38] "GET /books HTTP/1.1" 200 -
192.168.56.1 - - [15/Mar/2025 21:00:38] "GET /books HTTP/1.1" 200 -


Error: Attempt to add duplicate book


## A reset script that runs before each test

The file `reset.py` is a simple Python script that resets the system under test (SUT) to a known state. We will use Provengoe's `--before` command line argument to run this script before each test.


In [38]:
import platform


# Write the reset.py script
with open("reset.py", "w") as f:
    f.write(f"""\
import requests
test_data = {{
    'users': [{{ 'id': 1, 'name': 'Test User' }}],
    'books': {{'1': {{ 'id': '1', 'title': 'Test Book' }}}}
}}
requests.post('{url}/reset', json=test_data)
""")

# A hack to run reset on linux because the space in "python reset.py" is not allowed
if platform.system() != "Windows":
    !echo '#!/bin/bash' > before_script.sh
    !echo 'python reset.py' >> before_script.sh
    !chmod +x before_script.sh

# Test model

## Creating a new empty model

We first create a new test model called `BookStoreDemo` by running the following command:


In [39]:
!provengo --batch-mode create BookStoreDemo

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

[SETUP] INFO We'd love to hear from you! visit https://provengo.tech/feedback/
[SETUP] ERR c:\temp\ProvengoDemo\BookStoreDemo Folder already exist. delete the existing folder or create new project with new name.
[Main ] ERR Terminating because of previous errors. See logs.
[Main ] ERR 40 BadRequest


## Adding file containing general test information

An effective way to set gloabl parameters to the testing model is to use a file that contains general test information. This file can be imported into the test model and used to set global parameters. In this example, we will use a file called `_constatnts.py` to set the global parameters `host` and `port`. This parameters will be available to all our scriptss.

In [40]:
with open("BookStoreDemo/spec/js/_constants.js", "w") as f:
  f.write(f"const {host=}; const {port=};")

## Adding interface function to simplify the test scripts

We will add a file called `interface.py` that contains functions that will be used to simplify the test scripts. This function will be used to make HTTP requests to the SUT. These are just simple wrapper function that hide the unneccesary details of making HTTP requests.

In [41]:
%%writefile BookStoreDemo//spec//js//interface.js

const svc = new RESTSession("http://" + host + ":" + port, "provengo basedclient", {
    headers: { "Content-Type": "application/json" }
});

// Add a book to the store
function addBook(id, title) {
    svc.post("/books", { body: JSON.stringify({ id: id, title: title, }), });
}

// Add a user to the store
function addUser(id, name) {
    svc.post("/users", { body: JSON.stringify({ id: id, name: name, }), });
}

// Try to add a book that already exists
function tryToAddExistingBook(id, title) {
    svc.post("/books", {
        body: JSON.stringify({ id: id, title: title }),
        expectedResponseCodes: [400]
    });
}

// Try to add a user that already exists
function tryToAddExistingUser(id, name) {
    svc.post("/users", {
        body: JSON.stringify({ id: id, name: name }),
        expectedResponseCodes: [400]
    });
}

// Verify that a user exists
function verifyUserExists(id, name) {
    svc.get("/users", {
        callback: function (response) {
            user = JSON.parse(response.body);
            for (let i = 0; i < user.length; i++) {
                if (user[i].id === id && user[i].name === name) {
                    return pvg.success("User exists");
                }
            }
            return pvg.fail("Expected a user to exists but it does not");
        }
    });
}

// Verify that a user does not exist
function verifyUserDoesNotExist(id, name) {
    svc.get("/users", {
        callback: function (response) {
            user = JSON.parse(response.body);
            for (let i = 0; i < user.length; i++) {
                if (user[i].id === id && user[i].name === name) {
                    return pvg.fail("Expected a user to not exist but it does");
                }
            }
            return pvg.success("User does not exist");
        }
    });
}


// Verify that a book exists
function verifyBookExists(id, title) {
    svc.get("/books", {
        callback: function (response) {
            book = JSON.parse(response.body);
            for (let i = 0; i < book.length; i++) {
                if (book[i].id === id && book[i].title === title) {
                    return pvg.success("Book exists");
                }
            }
            return pvg.fail("Expected a book to exists but it does not");
        }
    });
}

// Verify that a book does not exist
function verifyBookDoesNotExist(id, title) {
    svc.get("/books", {
        callback: function (response) {
            book = JSON.parse(response.body);
            for (let i = 0; i < book.length; i++) {
                if (book[i].id === id && book[i].title === title) {
                    return pvg.fail("Expected a book to not exist but it does");
                }
            }
            return pvg.success("Book does not exist");
        }
    });
}

Overwriting BookStoreDemo//spec//js//interface.js


## Writing our first (linear) test scripts

Our first test script is a standard sequence of HTTP requests that tests the basic functionality of the SUT. The script,  called `hello_world.js`, is shown below:

In [48]:
%%writefile BookStoreDemo/spec/js/hello-world.js

bthread("User API", function () {
  addUser(111, "John Doe");
  tryToAddExistingUser(111, "John Doe");
  verifyUserExists(111, "John Doe");
  verifyUserDoesNotExist(222, "Jane Doe");
  addUser(222, "John Doe");
  verifyUserExists(222, "John Doe");
});

Overwriting BookStoreDemo/spec/js/hello-world.js


In [49]:
if platform.system() == "Windows":
    !provengo run BookStoreDemo --before="python reset.py" 
else:
    !provengo run BookStoreDemo --before=./before_script.sh

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

[SETUP] INFO We'd love to hear from you! visit https://provengo.tech/feedback/
[SETUP] INFO Input path: c:\temp\ProvengoDemo\BookStoreDemo
[RUN  ] INFO Preparing to run the model
[RUN>BUILD] INFO interface.js: Library 'REST' summoned automatically. Add "//@provengo summon rest" to the top of the file to explicitly summon it. Set the auto-summon configuration key to false to disable this behavior
[RUN>random>Before Test] INFO Running Before Test command: python reset.py
[RUN>random>Before Test] INFO Done running Before Test command: python reset.py
[RUN>random] INFO B-program started
[RUN>random] INFO Selected: [POST {JS_Obj lib:"REST", method:"POST", url:"http

We can create other scripts that test other parts of the SUT. For example, we can test book iterfae as shown below:

In [58]:
%%writefile BookStoreDemo/spec/js/hello-world.js
bthread("Book API", function () {
    addBook(111, "The Great Gatsby"); 
    tryToAddExistingBook(111, "The Great Gatsby");
    addBook(222, "The Catcher in the Rye");
    addBook(333, "The Catcher in the Rye");
    verifyBookExists(111, "The Great Gatsby");
    verifyBookExists(222, "The Catcher in the Rye");
    verifyBookDoesNotExist(444, "Harry Potter");
});


Overwriting BookStoreDemo/spec/js/hello-world.js


In [59]:
if platform.system() == "Windows":
    !provengo run BookStoreDemo --before="python reset.py" 
else:
    !provengo run BookStoreDemo --before=./before_script.sh

        / /
     /\/ /   ____
  /\/ /\/ / |  _ \ _ __  _____   __ ___  _ __   __ _  ___
 /  \/\/\/  | |_) | '__|/ _ \ \ / // _ \| '_ \ / _` |/ _ \
 \  /\/\/\  |  __/| |  | (_) \ V /|  __/| | | | (_| | (_) |
  \/\ \/\ \ |_|   |_|   \___/ \_/  \___||_| |_|\__, |\___/
     \/\ \                                     |___/
        \ \

[SETUP] INFO We'd love to hear from you! visit https://provengo.tech/feedback/
[SETUP] INFO Input path: c:\temp\ProvengoDemo\BookStoreDemo
[RUN  ] INFO Preparing to run the model
[RUN>BUILD] INFO interface.js: Library 'REST' summoned automatically. Add "//@provengo summon rest" to the top of the file to explicitly summon it. Set the auto-summon configuration key to false to disable this behavior
[RUN>random>Before Test] INFO Running Before Test command: python reset.py
[RUN>random>Before Test] INFO Done running Before Test command: python reset.py
[RUN>random] INFO B-program started
[RUN>random] INFO Selected: [POST {JS_Obj lib:"REST", method:"POST", url:"http

## Interleaving two scenarios

In [ ]:
%%writefile BookStoreDemo/spec/js/hello-world.js

bthread("Book API", function () {
    addBook(111, "The Great Gatsby"); 
    tryToAddExistingBook(111, "The Great Gatsby");
    addBook(222, "The Catcher in the Rye");
    addBook(333, "The Catcher in the Rye");
    verifyBookExists(111, "The Great Gatsby");
    verifyBookExists(222, "The Catcher in the Rye");
    verifyBookDoesNotExist(444, "Harry Potter");
});


bthread("User API", function () {
  addUser(111, "John Doe");
  tryToAddExistingUser(111, "John Doe");
  verifyUserExists(111, "John Doe");
  verifyUserDoesNotExist(222, "Jane Doe");
  addUser(222, "John Doe");
  verifyUserExists(222, "John Doe");
});

In [ ]:
if platform.system() == "Windows":
    !provengo run BookStoreDemo --before="python reset.py" 
else:
    !provengo run BookStoreDemo --before=./before_script.sh